**1. Autoencoders**

* **Definisi dan Tujuan:** *Autoencoders* adalah jaringan saraf tiruan yang mampu mempelajari representasi padat (disebut *latent representations* atau *codings*) dari data input tanpa pengawasan. *Codings* ini biasanya memiliki dimensionalitas yang jauh lebih rendah daripada data input, sehingga *autoencoders* berguna untuk reduksi dimensionalitas, terutama untuk tujuan visualisasi. *Autoencoders* juga berfungsi sebagai detektor fitur dan dapat digunakan untuk *unsupervised pretraining* pada *deep neural networks*. Beberapa *autoencoders* adalah model generatif, yang mampu menghasilkan data baru secara acak yang sangat mirip dengan data pelatihan. Namun, gambar yang dihasilkan biasanya buram dan kurang realistis.
* **Cara Kerja:** *Autoencoders* belajar untuk menyalin input mereka ke output mereka. Hal ini menjadi sulit dengan membatasi jaringan, misalnya, dengan membatasi ukuran *latent representations* atau dengan menambahkan *noise* ke input dan melatih jaringan untuk memulihkan input asli. Batasan ini memaksa *autoencoder* untuk mempelajari cara yang efisien dalam merepresentasikan data.
* **Arsitektur:** Sebuah *autoencoder* selalu terdiri dari dua bagian: sebuah *encoder* (atau *recognition network*) yang mengubah input menjadi *latent representation*, diikuti oleh sebuah *decoder* (atau *generative network*) yang mengubah *internal representation* menjadi output. Output sering disebut *reconstructions* karena *autoencoder* mencoba untuk merekonstruksi input, dan *cost function* berisi *reconstruction loss* yang menghukum model ketika *reconstructions* berbeda dari input.
* **Jenis-jenis Autoencoder:**
    * **Undercomplete Autoencoders:** *Autoencoder* yang *internal representation*-nya memiliki dimensionalitas yang lebih rendah daripada data input. Ini memaksa *autoencoder* untuk mempelajari fitur paling penting dalam data input dan menghilangkan yang tidak penting.
        * **PCA dengan Undercomplete Linear Autoencoder:** Jika *autoencoder* hanya menggunakan fungsi aktivasi linier dan *cost function*-nya adalah *mean squared error* (MSE), maka ia akan melakukan *Principal Component Analysis* (PCA).
        * **Implementasi:**
            ```python
            encoder = keras.models.Sequential([keras.layers.Dense(2, input_shape=[3])])
            decoder = keras.models.Sequential([keras.layers.Dense(3, input_shape=[2])])
            autoencoder = keras.models.Sequential([encoder, decoder])

            autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.1))
            history = autoencoder.fit(X_train, X_train, epochs=20)
            codings = encoder.predict(X_train)
            ```
            Kode ini membuat *linear autoencoder* sederhana untuk melakukan PCA pada *dataset* 3D, memproyeksikannya ke 2D. *Encoder* dan *decoder* adalah model `Sequential` dengan satu lapisan `Dense` masing-masing, dan *autoencoder* adalah model `Sequential` yang berisi *encoder* diikuti oleh *decoder*. Output *autoencoder* sama dengan jumlah input, dan tidak ada fungsi aktivasi yang digunakan (semua neuron linier) dengan MSE sebagai *cost function*.
    * **Stacked Autoencoders (Deep Autoencoders):** *Autoencoders* dengan beberapa *hidden layers*. Menambahkan lebih banyak lapisan membantu *autoencoder* mempelajari *codings* yang lebih kompleks. Arsitektur biasanya simetris sehubungan dengan *central hidden layer* (*coding layer*).
        * **Implementasi:**
            ```python
            stacked_encoder = keras.models.Sequential([
                keras.layers.Flatten(input_shape=[28, 28]),
                keras.layers.Dense(100, activation="selu"),
                keras.layers.Dense(30, activation="selu"),
            ])

            stacked_decoder = keras.models.Sequential([
                keras.layers.Dense(100, activation="selu", input_shape=[30]),
                keras.layers.Dense(28*28, activation="sigmoid"),
                keras.layers.Reshape([28, 28])
            ])

            stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])

            stacked_ae.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr=1.5))
            history = stacked_ae.fit(X_train, X_train, epochs=10, validation_data=[X_valid, X_valid])
            ```
            Kode ini membangun *stacked autoencoder* untuk Fashion MNIST. *Encoder* meratakan gambar 28x28 piksel menjadi vektor 784, lalu memprosesnya melalui dua lapisan `Dense` dengan ukuran yang semakin kecil (100 unit kemudian 30 unit) menggunakan fungsi aktivasi SELU. *Decoder* mengambil *codings* ukuran 30, memprosesnya melalui dua lapisan `Dense` dengan ukuran yang semakin besar (100 unit kemudian 784 unit), dan membentuk kembali vektor akhir menjadi larik 28x28. *Binary cross-entropy loss* digunakan karena tugas rekonstruksi diperlakukan sebagai masalah klasifikasi biner multilabel.
        * **Tying Weights:** Teknik umum untuk *autoencoder* yang simetris adalah mengikat bobot lapisan *decoder* ke bobot lapisan *encoder*. Ini mengurangi setengah jumlah bobot dalam model, mempercepat pelatihan, dan membatasi risiko *overfitting*.
        * **Training One Autoencoder at a Time (Greedy Layerwise Training):** Melatih satu *shallow autoencoder* pada satu waktu, lalu menumpuk semuanya menjadi satu *stacked autoencoder*.
    * **Convolutional Autoencoders:** Digunakan untuk gambar. *Encoder* adalah CNN biasa yang terdiri dari *convolutional layers* dan *pooling layers*, yang mengurangi dimensionalitas spasial sambil meningkatkan kedalaman. *Decoder* melakukan hal sebaliknya menggunakan *transpose convolutional layers*.
        * **Implementasi:**
            ```python
            conv_encoder = keras.models.Sequential([
                keras.layers.Reshape([28, 28, 1], input_shape=[28, 28]),
                keras.layers.Conv2D(16, kernel_size=3, padding="same", activation="selu"),
                keras.layers.MaxPool2D(pool_size=2),
                keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="selu"),
                keras.layers.MaxPool2D(pool_size=2),
                keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="selu"),
                keras.layers.MaxPool2D(pool_size=2)
            ])

            conv_decoder = keras.models.Sequential([
                keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding="valid",
                                            activation="selu", input_shape=[3, 3, 64]),
                keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, padding="same",
                                            activation="selu"),
                keras.layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding="same",
                                            activation="sigmoid"),
                keras.layers.Reshape([28, 28])
            ])

            conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
            ```
            *Convolutional autoencoder* ini untuk Fashion MNIST. *Encoder* menggunakan lapisan `Conv2D` dan `MaxPool2D` untuk mengurangi dimensi gambar. *Decoder* menggunakan lapisan `Conv2DTranspose` untuk mengembalikan dimensi gambar ke ukuran aslinya.
    * **Recurrent Autoencoders:** Digunakan untuk urutan, seperti *time series* atau teks. *Encoder* biasanya RNN *sequence-to-vector* yang mengompresi *input sequence* menjadi satu vektor. *Decoder* adalah RNN *vector-to-sequence* yang melakukan hal sebaliknya.
        * **Implementasi:**
            ```python
            recurrent_encoder = keras.models.Sequential([
                keras.layers.LSTM(100, return_sequences=True, input_shape=[None, 28]),
                keras.layers.LSTM(30)
            ])

            recurrent_decoder = keras.models.Sequential([
                keras.layers.RepeatVector(28, input_shape=[30]),
                keras.layers.LSTM(100, return_sequences=True),
                keras.layers.TimeDistributed(keras.layers.Dense(28, activation="sigmoid"))
            ])

            recurrent_ae = keras.models.Sequential([recurrent_encoder, recurrent_decoder])
            ```
            *Recurrent autoencoder* ini dapat memproses urutan dengan panjang berapa pun, dengan 28 dimensi per langkah waktu.
    * **Denoising Autoencoders:** Memaksa *autoencoder* untuk mempelajari fitur yang berguna dengan menambahkan *noise* ke inputnya, melatihnya untuk memulihkan input asli yang bebas *noise*. *Noise* bisa berupa *Gaussian noise* murni atau input yang dimatikan secara acak (mirip *dropout*).
        * **Implementasi:**
            ```python
            dropout_encoder = keras.models.Sequential([
                keras.layers.Flatten(input_shape=[28, 28]),
                keras.layers.Dropout(0.5),
                keras.layers.Dense(100, activation="selu"),
                keras.layers.Dense(30, activation="selu")
            ])

            dropout_decoder = keras.models.Sequential([
                keras.layers.Dense(100, activation="selu", input_shape=[30]),
                keras.layers.Dense(28*28, activation="sigmoid"),
                keras.layers.Reshape([28, 28])
            ])

            dropout_ae = keras.models.Sequential([dropout_encoder, dropout_decoder])
            ```
            Ini adalah *stacked autoencoder* biasa dengan lapisan `Dropout` tambahan yang diterapkan pada input *encoder*.
    * **Sparse Autoencoders:** Menambahkan *term* yang sesuai ke *cost function* untuk mendorong *autoencoder* mengurangi jumlah neuron aktif di *coding layer*. Hal ini memaksa *autoencoder* untuk merepresentasikan setiap input sebagai kombinasi sejumlah kecil aktivasi.
        * **L1 Regularization:** Menggunakan fungsi aktivasi *sigmoid* di *coding layer* dan menambahkan *l1 regularization* ke aktivasi *coding layer*.
        * **Kullback-Leibler (KL) Divergence:** Mengukur *sparsity* aktual dari *coding layer* pada setiap iterasi pelatihan dan menghukum model ketika *sparsity* yang diukur berbeda dari *target sparsity*. Divergensi KL memiliki *gradients* yang jauh lebih kuat daripada *mean squared error*.
        * **Formula KL Divergence:**
            $D_{KL}(P||Q)=\sum_{i}P(i)log\frac{P(i)}{Q(i)}$
            Untuk *target sparsity* $p$ dan *actual sparsity* $q$:
            $D_{KL}(p||q)=p~log\frac{p}{q}+(1-p)log\frac{1-p}{1-q}$
        * **Implementasi KL Divergence Regularizer:**
            ```python
            K = keras.backend
            kl_divergence = keras.losses.kullback_leibler_divergence

            class KLDivergenceRegularizer(keras.regularizers.Regularizer):
                def __init__(self, weight, target=0.1):
                    self.weight = weight
                    self.target = target
                    super().__init__()

                def __call__(self, inputs):
                    mean_activities = K.mean(inputs, axis=0)
                    return self.weight * (
                        kl_divergence(self.target, mean_activities) +
                        kl_divergence(1. - self.target, 1. - mean_activities))
            ```
            Ini adalah *custom regularizer* untuk menerapkan *KL divergence regularization*.
    * **Variational Autoencoders (VAEs):** *Probabilistic autoencoders* (outputnya sebagian ditentukan oleh peluang) dan *generative autoencoders* (dapat menghasilkan instance baru yang terlihat seperti sampel dari *training set*). *Encoder* menghasilkan *mean coding* ($\mu$) dan *standard deviation* ($\sigma$), dan *coding* sebenarnya diambil secara acak dari distribusi Gaussian dengan *mean* $\mu$ dan *standard deviation* $\sigma$.
        * **Cost Function VAE:** Terdiri dari dua bagian: *reconstruction loss* (seperti biasa) dan *latent loss* yang mendorong *codings* agar terlihat seperti sampel dari distribusi Gaussian.
        * **Formula Latent Loss:**
            $\Theta=-\frac{1}{2}\sum_{i=1}^{K}1+log(\sigma_{i}^{2})-\sigma_{i}^{2}-\mu_{i}^{2}$
            Atau dengan *tweak* $\gamma=log(\sigma^{2})$:
            $\mathcal{Z}=-\frac{1}{2}\sum_{i=1}^{K}1+y_{i}-exp(y_{i})-\mu_{i}^{2}$
        * **Implementasi Sampling Layer:**
            ```python
            class Sampling(keras.layers.Layer):
                def call(self, inputs):
                    mean, log_var = inputs
                    return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean
            ```
            Lapisan `Sampling` ini mengambil `mean` ($\mu$) dan `log_var` ($\gamma$) sebagai input. Ia menggunakan `K.random_normal()` untuk mengambil sampel vektor acak dari distribusi Normal dengan *mean* 0 dan *standard deviation* 1, lalu mengalikannya dengan `exp(log_var / 2)` (yang sama dengan $\sigma$) dan menambahkan `mean` untuk menghasilkan vektor *codings* yang diambil sampelnya dari distribusi Normal dengan *mean* $\mu$ dan *standard deviation* $\sigma$.
        * **Implementasi VAE:**
            ```python
            codings_size = 10
            inputs = keras.layers.Input(shape=[28, 28])
            z = keras.layers.Flatten()(inputs)
            z = keras.layers.Dense(150, activation="selu")(z)
            z = keras.layers.Dense(100, activation="selu")(z)
            codings_mean = keras.layers.Dense(codings_size)(z)
            codings_log_var = keras.layers.Dense(codings_size)(z)
            codings = Sampling()([codings_mean, codings_log_var])
            variational_encoder = keras.Model(
                inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

            decoder_inputs = keras.layers.Input(shape=[codings_size])
            x = keras.layers.Dense(100, activation="selu")(decoder_inputs)
            x = keras.layers.Dense(150, activation="selu")(x)
            x = keras.layers.Dense(28*28, activation="sigmoid")(x)
            outputs = keras.layers.Reshape([28, 28])(x)
            variational_decoder = keras.Model(inputs=[decoder_inputs], outputs=[outputs])

            _, _, codings = variational_encoder(inputs)
            reconstructions = variational_decoder(codings)
            variational_ae = keras.Model(inputs=[inputs], outputs=[reconstructions])

            latent_loss = -0.5 * K.sum(
                1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
                axis=-1)
            variational_ae.add_loss(K.mean(latent_loss) / 784.)
            variational_ae.compile(loss="binary_crossentropy", optimizer="rmsprop")
            ```
            Kode ini membuat *variational autoencoder* menggunakan *Functional API*. *Encoder* menghasilkan *mean* dan *log_var* dari *codings*, yang kemudian digunakan oleh lapisan `Sampling` untuk menghasilkan *codings* aktual. *Decoder* kemudian merekonstruksi gambar dari *codings*. *Latent loss* dan *reconstruction loss* ditambahkan ke model.

**2. Generative Adversarial Networks (GANs)**

* **Definisi dan Tujuan:** GANs diperkenalkan pada tahun 2014. Mereka terdiri dari dua jaringan saraf yang bersaing satu sama lain: sebuah *generator* dan sebuah *discriminator*. GANs sangat efektif dalam menghasilkan data baru yang sangat realistis, terutama gambar.
* **Komponen GAN:**
    * **Generator:** Mengambil distribusi acak sebagai input (biasanya Gaussian) dan menghasilkan beberapa data, biasanya gambar. Input acak dapat dianggap sebagai *latent representations* atau *codings* dari gambar yang akan dihasilkan. Tujuannya adalah untuk "menipu" *discriminator*.
    * **Discriminator:** Mengambil gambar palsu dari *generator* atau gambar asli dari *training set* sebagai input, dan harus menebak apakah gambar input itu palsu atau asli. Tujuannya adalah untuk "memberi tahu" mana yang palsu dan mana yang asli.
* **Proses Pelatihan:**
    * **Fase 1 (Melatih Discriminator):** Sebuah *batch* gambar asli diambil dari *training set* dan dilengkapi dengan jumlah gambar palsu yang sama yang dihasilkan oleh *generator*. Label diatur ke 0 untuk gambar palsu dan 1 untuk gambar asli. *Discriminator* dilatih pada *batch* berlabel ini untuk satu langkah, menggunakan *binary cross-entropy loss*. Selama fase ini, hanya bobot *discriminator* yang dioptimalkan.
    * **Fase 2 (Melatih Generator):** *Generator* digunakan untuk menghasilkan *batch* gambar palsu lainnya. *Discriminator* digunakan untuk memberi tahu apakah gambar-gambar ini palsu atau asli. Kali ini, semua label diatur ke 1 (asli) karena tujuan *generator* adalah membuat *discriminator* percaya bahwa gambar-gambar palsu itu asli. Bobot *discriminator* dibekukan selama langkah ini, sehingga *backpropagation* hanya memengaruhi bobot *generator*.
* **Implementasi Sederhana GAN:**
    ```python
    codings_size = 30
    generator = keras.models.Sequential([
        keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
        keras.layers.Dense(150, activation="selu"),
        keras.layers.Dense(28*28, activation="sigmoid"),
        keras.layers.Reshape([28, 28])
    ])

    discriminator = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(150, activation="selu"),
        keras.layers.Dense(100, activation="selu"),
        keras.layers.Dense(1, activation="sigmoid")
    ])

    gan = keras.models.Sequential([generator, discriminator])

    discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
    discriminator.trainable = False
    gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

    batch_size = 32
    dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
    dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

    def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
        generator, discriminator = gan.layers
        for epoch in range(n_epochs):
            for X_batch in dataset:
                # phase 1 - training the discriminator
                noise = tf.random.normal(shape=[batch_size, codings_size])
                generated_images = generator(noise)
                X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
                y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
                discriminator.trainable = True
                discriminator.train_on_batch(X_fake_and_real, y1)

                # phase 2 - training the generator
                noise = tf.random.normal(shape=[batch_size, codings_size])
                y2 = tf.constant([[1.]] * batch_size)
                discriminator.trainable = False
                gan.train_on_batch(noise, y2)

    train_gan(gan, dataset, batch_size, codings_size)
    ```
    *Generator* mirip dengan *decoder autoencoder*, dan *discriminator* adalah *binary classifier*. *Binary cross-entropy loss* digunakan untuk *discriminator* dan model GAN. Selama fase pelatihan *generator*, *discriminator* dibuat *non-trainable*.
* **Kesulitan dalam Melatih GANs:**
    * **Mode Collapse:** Output *generator* secara bertahap menjadi kurang beragam. *Generator* mungkin menjadi sangat baik dalam menghasilkan satu jenis output (misalnya, sepatu) dan melupakan cara menghasilkan jenis lainnya.
    * **Ketidakstabilan Pelatihan:** Parameter *generator* dan *discriminator* dapat berosilasi dan menjadi tidak stabil. Pelatihan dapat tiba-tiba menyimpang tanpa alasan yang jelas.
    * **Sensitivitas Hiperparameter:** GANs sangat sensitif terhadap *hyperparameter*.
* **Teknik untuk Mengatasi Kesulitan:**
    * **Experience Replay:** Menyimpan gambar yang dihasilkan oleh *generator* pada setiap iterasi dalam *replay buffer* dan melatih *discriminator* menggunakan gambar asli ditambah gambar palsu dari *buffer* ini. Ini mengurangi kemungkinan *discriminator* melakukan *overfit* pada output *generator* terbaru.
    * **Mini-batch Discrimination:** Mengukur seberapa mirip gambar di seluruh *batch* dan memberikan *statistic* ini ke *discriminator*, sehingga dapat dengan mudah menolak seluruh *batch* gambar palsu yang kurang beragam. Ini mendorong *generator* untuk menghasilkan variasi gambar yang lebih besar, mengurangi risiko *mode collapse*.
* **Arsitektur GAN yang Berhasil:**
    * **Deep Convolutional GANs (DCGANs):** DCGANs menggunakan pedoman tertentu untuk membangun GANs *convolutional* yang stabil: mengganti *pooling layers* dengan *strided convolutions* (di *discriminator*) dan *transposed convolutions* (di *generator*), menggunakan *Batch Normalization* (kecuali di lapisan output *generator* dan lapisan input *discriminator*), menghapus *fully connected hidden layers* untuk arsitektur yang lebih dalam, menggunakan aktivasi ReLU di *generator* (kecuali lapisan output yang menggunakan tanh), dan menggunakan aktivasi *leaky ReLU* di *discriminator*.
        * **Implementasi DCGAN:**
            ```python
            codings_size = 100
            generator = keras.models.Sequential([
                keras.layers.Dense(7 * 7 * 128, input_shape=[codings_size]),
                keras.layers.Reshape([7, 7, 128]),
                keras.layers.BatchNormalization(),
                keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same",
                                            activation="selu"),
                keras.layers.BatchNormalization(),
                keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="same",
                                            activation="tanh")
            ])

            discriminator = keras.models.Sequential([
                keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="same",
                                    activation=keras.layers.LeakyReLU(0.2),
                                    input_shape=[28, 28, 1]),
                keras.layers.Dropout(0.4),
                keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="same",
                                    activation=keras.layers.LeakyReLU(0.2)),
                keras.layers.Dropout(0.4),
                keras.layers.Flatten(),
                keras.layers.Dense(1, activation="sigmoid")
            ])

            gan = keras.models.Sequential([generator, discriminator])
            ```
            *Generator* memproyeksikan *codings* ke tensor yang kemudian di-*upsample* oleh *transpose convolutional layers*. *Discriminator* menggunakan *strided convolutions* alih-alih *max pooling* dan fungsi aktivasi *leaky ReLU*.
        * **Conditional GAN (CGAN):** Menambahkan kelas gambar sebagai input tambahan ke *generator* dan *discriminator*, memungkinkan kontrol atas kelas gambar yang dihasilkan.
    * **Progressive Growing of GANs:** Menghasilkan gambar kecil di awal pelatihan, lalu secara bertahap menambahkan *convolutional layers* ke *generator* dan *discriminator* untuk menghasilkan gambar yang semakin besar. Teknik ini menggunakan *fade-in/fade-out* saat menambahkan lapisan baru.
        * **Minibatch Standard Deviation Layer:** Ditambahkan di dekat akhir *discriminator*, menghitung *standard deviation* di seluruh *channel* dan *instance* dalam *batch*. Ini membantu *discriminator* mendeteksi kurangnya variasi dalam gambar yang dihasilkan, mendorong *generator* untuk menghasilkan output yang lebih beragam.
        * **Equalized Learning Rate:** Menginisialisasi semua bobot menggunakan distribusi Gaussian sederhana, tetapi menskalakan bobot ke bawah saat *runtime*. Ini memastikan bahwa *dynamic range* sama untuk semua parameter, mempercepat dan menstabilkan pelatihan.
        * **Pixelwise Normalization Layer:** Ditambahkan setelah setiap *convolutional layer* di *generator*. Ini menormalisasi setiap aktivasi berdasarkan semua aktivasi di gambar yang sama dan lokasi yang sama, tetapi di semua *channel*. Teknik ini menghindari ledakan dalam aktivasi karena persaingan yang berlebihan antara *generator* dan *discriminator*.
    * **StyleGANs:** Memajukan *state of the art* dalam generasi gambar resolusi tinggi menggunakan teknik *style transfer* di *generator*.
        * **Mapping Network:** Sebuah MLP delapan lapisan yang memetakan *latent representations* $z$ ke vektor $w$. Vektor ini kemudian dikirim melalui beberapa transformasi *affine* untuk menghasilkan banyak vektor yang mengontrol *style* gambar yang dihasilkan pada tingkat yang berbeda.
        * **Synthesis Network:** Bertanggung jawab untuk menghasilkan gambar. Ia memiliki input *constant learned* dan memprosesnya melalui beberapa lapisan *convolutional* dan *upsampling*. *Noise* ditambahkan ke input dan semua output lapisan *convolutional*. Setiap lapisan *noise* diikuti oleh lapisan *Adaptive Instance Normalization* (AdaIN) yang menstandardisasi setiap *feature map* secara independen dan menggunakan vektor *style* untuk menentukan skala dan offset setiap *feature map*.
        * **Mixing Regularization (Style Mixing):** Persentase gambar yang dihasilkan diproduksi menggunakan dua *codings* yang berbeda. Hal ini mencegah jaringan berasumsi bahwa *style* pada tingkat yang berdekatan berkorelasi, mendorong *locality* dalam GAN.